# Random Forest classifier with pipeline and hyperparameter tuning

In [104]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [105]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [106]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [107]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [108]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [109]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [110]:
df['time'] = df['time'].map({'Dinner' : 0, 'Lunch': 1})

In [111]:
df['time'].unique()

[0, 1]
Categories (2, int64): [1, 0]

In [112]:
df['time'] = df['time'].astype('int64')

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int64   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int64(2)
memory usage: 8.9 KB


In [114]:
# divide dataset into dependent- independent features

In [115]:
X = df.drop(['time'], axis=1)
y = df['time']

In [116]:
# train-test split

In [117]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

### A machine learning pipeline is a series of steps that automate the process of building, training, evaluating, and deploying machine learning models.

In [118]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #handle missing values
from sklearn.preprocessing import StandardScaler # feature scaling
from sklearn.preprocessing import OneHotEncoder # feature encoding
from sklearn.compose import ColumnTransformer # used to combine multiple pipeline

In [119]:
categorical_cols = ['sex', 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

### we will create seperate pipeline for categorcal as well as numerical columns

In [123]:
# automate feature engineering 

# numerical pipeline
num_pipeline = Pipeline(
steps = [
    ('imputer', SimpleImputer(strategy='median')),#missing vals
    ('sacler', StandardScaler())#feature scaling
])

# categorical pipeline
cat_pipeline = Pipeline(
steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehotencoder', OneHotEncoder())
])

### to combine both categorical and numerical pipeline we will use column transformer

In [124]:
preprocessor = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('sacler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [127]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

### model traning automation

In [148]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB

In [150]:
models = {
    'Random Forest' : RandomForestClassifier(),
    'Logistic Regression' : LogisticRegression(),
    'DecisionTreeClassifier' : DecisionTreeClassifier(),
    'naive_bayes' : BernoulliNB()
}

In [151]:
from sklearn.metrics import accuracy_score

In [156]:
# func to automate model training and model evaluation

def evaluate_model(X_train,y_train,X_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy score for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report

In [155]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'DecisionTreeClassifier': 0.9387755102040817,
 'naive_bayes': 0.9795918367346939}

In [161]:
classifier = RandomForestClassifier()

In [157]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [158]:
from sklearn.model_selection import RandomizedSearchCV

In [163]:
cv = RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [164]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.974 total time=   0.9s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.923 total time=   0.9s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=200;, score=1.000 total time=   0.8s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total time=   0.8s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.923 total time=   0.9s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.949 total time=   0.4s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [165]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'gini'}